# Advent of Code 2019 Python Solutions

Samuel Mignot's solution for [the 2019 Advent of Code challenge](https://adventofcode.com/). 

Have yet to get through the entire challenge, but slowly working my way throuh it.
The intcode computer, which is created and used in multiple challenges, is abstracted into an intcomp.py file.



## Global Setup

Handles imports, notebook magic, and notebook-wide constants

### imports

In [7]:
import string
import numpy as np
from itertools import cycle
import requests
import collections
from collections import deque
from pprint import pprint
import operator
from time import sleep
import itertools
import re
import functools
from functools import reduce
from dataclasses import dataclass
from copy import deepcopy
import networkx as nx
import matplotlib.pyplot as plt
import intcomp

### notebook magic 

In [8]:
%matplotlib widget

### constants

In [9]:
LOWERCASE = string.ascii_lowercase
UPPERCASE = string.ascii_uppercase

### helpers

In [10]:
def get_level_input(lvl_num):
    with open(f"advent_inputs/{lvl_num}.txt") as f:
        level_input=f.read()
        return level_input
    
def print_result(answer):
    pprint("RESULT: "+str(answer))
    print()
    pprint("TIME"+"."*60)
    
class StopExecution(Exception):
    def _render_traceback_(self):
        pass

## Day 1: The Tyranny of the Rocket Equation

### setup

In [11]:
module_weights = get_level_input("01").splitlines()
module_weights = list(map(int, module_weights))

### part one

In [12]:
%%time
total_weight = sum([x//3 - 2 for x in module_weights])
print_result(total_weight)

'RESULT: 3432671'

'TIME............................................................'
CPU times: user 128 µs, sys: 34 µs, total: 162 µs
Wall time: 145 µs


### part two

In [13]:
%%time

def weight_of_weight_gen(m_weight):
    while m_weight>0:
        yield m_weight
        m_weight = m_weight//3 - 2
    
def get_req_fuel(m_weight): 
    return sum([i for i in weight_of_weight_gen(m_weight)][1:])
    
total_weight = sum([get_req_fuel(x) for x in module_weights])
print_result(total_weight)

'RESULT: 5146132'

'TIME............................................................'
CPU times: user 373 µs, sys: 47 µs, total: 420 µs
Wall time: 397 µs


## Day 2: 1202 Program Alarm

### setup

In [14]:
comp_string = get_level_input("02")
comp_string = list(map(int, comp_string.split(',')))

### part one

In [15]:
%%time

comp_string_prog = comp_string[:]
comp_string_prog[1] = 12
comp_string_prog[2] = 2
res = intcomp.run_computer(comp_string_prog, [])
print_result(comp_string_prog[0])

'RESULT: 5290681'

'TIME............................................................'
CPU times: user 582 µs, sys: 271 µs, total: 853 µs
Wall time: 608 µs


### part two 

In [ ]:
%%time

TARGET = 19690720
NOUN_POS = 1
VERB_POS = 2
    
def find_noun_verb(comp_string):
    for noun in range(100):
        for verb in range(100):
            comp_string_copy = comp_string[:]
            comp_string_copy[NOUN_POS] = noun
            comp_string_copy[VERB_POS] = verb
            res = intcomp.run_computer(comp_string_copy, []) 
            if comp_string_copy[0] == -1: continue 
            if(comp_string_copy[0] == TARGET): return noun,verb

noun, verb = find_noun_verb(list(comp_string))

print_result(100 * noun + verb)

## Day 3: Crossed Wires 

### setup

In [11]:
wires_input = get_level_input("03").splitlines()
wires_input = [x.split(',') for x in wires_input]

X = 1
Y = 0
CENTER_POINT = (0,0)

def md(p1, p2):
    return (abs(p1[X]-p2[X]) + abs(p1[Y]-p2[Y]))

### part one

In [12]:
%%time

def generate_wire(instructions):
    wire = [CENTER_POINT]
    for x in instructions:
        cur_point = wire[-1]
        wire += {
         'U': [(cur_point[Y]-i, cur_point[X]) for i in range(1, int(x[1:])+1)],
         'D': [(cur_point[Y]+i, cur_point[X]) for i in range(1, int(x[1:])+1)],
         'L': [(cur_point[Y], cur_point[X]-i) for i in range(1, int(x[1:])+1)],
         'R': [(cur_point[Y], cur_point[X]+i) for i in range(1, int(x[1:])+1)]
        }[x[0]]
    return wire
                                  
    
wires = [set(generate_wire(w_i))-{CENTER_POINT} for w_i in wires_input]
res = min([md(x, CENTER_POINT) for x in set.intersection(*wires)])
print_result(res)

'RESULT: 225'

'TIME............................................................'
CPU times: user 302 ms, sys: 20.5 ms, total: 322 ms
Wall time: 323 ms


### part two

In [13]:
%%time

wires = [generate_wire(w_i) for w_i in wires_input]
intersections = set.intersection(*[set(wire) for wire in wires])-{CENTER_POINT}
res = min([wires[0].index(x)+wires[1].index(x) for x in intersections])
print_result(res)

'RESULT: 35194'

'TIME............................................................'
CPU times: user 449 ms, sys: 19.5 ms, total: 468 ms
Wall time: 470 ms


## Day 4: Secure Container

### setup

In [14]:
r_low, r_high = map(int, get_level_input("04").split("-"))

def check_value(val, functions):
    num_list = [int(i) for i in list(str(val))]
    return (all(f(num_list) for f in functions))

### part one

In [15]:
%%time

def check_increase(l):
    return all(l[i] <= l[i+1] for i in range(len(l)-1))

def check_double(l):
    return any(l[i] == l[i+1] for i in range(len(l)-1))

num_poss_pass = 0
for num in range(r_low, r_high+1):
    if(check_value(num, [check_increase, check_double])): num_poss_pass+=1

print_result(num_poss_pass)

'RESULT: 1694'

'TIME............................................................'
CPU times: user 1.74 s, sys: 10.6 ms, total: 1.75 s
Wall time: 1.76 s


### part two 

In [16]:
%%time

def check_increase(l):
    return all(l[i] <= l[i+1] for i in range(len(l)-1))

def check_ungrouped_double(l):
    l = [-1] + l +[-1]
    return any(l[i-1] != l[i] == l[i+1] != l[i+2] for i in range(1, len(l)-1))

num_poss_pass = 0
for num in range(r_low, r_high+1):
    num_list = [int(i) for i in list(str(num))]
    if(check_value(num, [check_increase, check_ungrouped_double])): num_poss_pass += 1

print_result(num_poss_pass)

'RESULT: 1148'

'TIME............................................................'
CPU times: user 2.6 s, sys: 13.9 ms, total: 2.61 s
Wall time: 2.63 s


## Day 5: Sunny with a Chance of Asteroids

### setup

In [17]:
comp_string = get_level_input("05")
comp_string = list(map(int, comp_string.split(',')))

### part one

In [18]:
%%time

_, outputs = intcomp.run_computer(comp_string[:], [1])
print_result(outputs[-1])

'RESULT: 10987514'

'TIME............................................................'
CPU times: user 288 µs, sys: 19 µs, total: 307 µs
Wall time: 297 µs


### part two 

In [19]:
%%time

_, outputs = intcomp.run_computer(comp_string[:], [5])
print_result(outputs[0])

'RESULT: 14195011'

'TIME............................................................'
CPU times: user 643 µs, sys: 156 µs, total: 799 µs
Wall time: 681 µs


## Day 6: Universal Orbit Map

### setup 

In [20]:
orbits = get_level_input("06").splitlines()
orbits = [orbit.split(')') for orbit in orbits]

### part one

In [21]:
%%time

orbit_graph = nx.DiGraph()
for orbit in orbits:
    orbit_graph.add_edge(*orbit)

orbit_checksum = sum([len(nx.descendants(orbit_graph, node)) for node in orbit_graph.nodes()])

print_result(orbit_checksum)

'RESULT: 144909'

'TIME............................................................'
CPU times: user 428 ms, sys: 5.79 ms, total: 434 ms
Wall time: 437 ms


### part two 

In [22]:
TARGET = "SAN"
START = "YOU"

orbit_graph = nx.Graph()
for orbit in orbits:
    orbit_graph.add_edge(*orbit)

START_SR = list(orbit_graph.neighbors(START))[0]
SAN_SR = list(orbit_graph.neighbors(TARGET))[0]

print_result(nx.shortest_path_length(orbit_graph, str(START_SR), str(SAN_SR)))
    

'RESULT: 259'

'TIME............................................................'


## Day 7: Amplification Circuit

### setup 

In [23]:
amp_prog = get_level_input("07")
amp_prog = list(map(int, amp_prog.split(',')))
NUM_AMPS = 5

### part one

In [24]:
%%time

amp_pos = list(itertools.permutations(list(range(NUM_AMPS))))

max_val = 0

for perm in amp_pos:
    inputs = [perm[0], 0]
    for i in range(NUM_AMPS):
        inputs = [perm[i+1] if i+1<NUM_AMPS else 0, intcomp.run_computer(amp_prog[:], inputs)[1][0]]
        if(inputs[1]>max_val):
            max_val = inputs[1]
        
print_result(max_val)

'RESULT: 21760'

'TIME............................................................'
CPU times: user 20.2 ms, sys: 1.01 ms, total: 21.2 ms
Wall time: 20.5 ms


### part two

In [25]:
%%time

amp_pos = list(itertools.permutations(list(range(NUM_AMPS, NUM_AMPS+5))))

signals = []
for perm in amp_pos:
    i = 0
    inputs = [[perm[j]] for j in range(NUM_AMPS)]
    inputs[0].append(0)
    last_e = 0
    amp_progs = [deepcopy(amp_prog) for j in range(NUM_AMPS)]
    ips = [0 for j in range(NUM_AMPS)]
    while(True):
        ips[i%NUM_AMPS], outputs = intcomp.run_computer(amp_progs[i%NUM_AMPS], inputs[i%NUM_AMPS], ip=ips[i%NUM_AMPS], return_on_output=True)
        if outputs == []:
            break
        if(i%NUM_AMPS == 4):
            last_e = outputs[0]
        i+=1
        inputs[i%NUM_AMPS].append(outputs[0])
    signals.append(last_e)
        
print_result(max(signals))

'RESULT: 69816958'

'TIME............................................................'
CPU times: user 252 ms, sys: 3.48 ms, total: 255 ms
Wall time: 256 ms


## Day 8: Space Image Format

### setup 

In [26]:
img = list(get_level_input("08"))
img = list(map(int, img))
IMG_W = 25
IMG_H = 6

### part one

In [27]:
%%time
arr_img = np.array(img)
lay_img = np.reshape(arr_img, (-1, IMG_H, IMG_W))

max_zero_index = np.argmin([(lay_img[i,:,:]==0).sum() for i in range(lay_img.shape[0])])

max_zero_layer = lay_img[max_zero_index, :, :]
NUM_1_BY_2 = (max_zero_layer == 1).sum() * (max_zero_layer == 2).sum()
print_result(NUM_1_BY_2)

'RESULT: 1360'

'TIME............................................................'
CPU times: user 2.36 ms, sys: 314 µs, total: 2.67 ms
Wall time: 2.46 ms


### part two

In [28]:
%%time
BLACK = 0 
WHITE = 1 
TRANS = 2

arr_img = np.array(img)
lay_img = np.reshape(arr_img, (-1, IMG_H, IMG_W))

final_image = np.full((IMG_H, IMG_W), 2)

spears = [lay_img[:, col, row] for col in range(lay_img.shape[1]) for row in range(lay_img.shape[2])]

first_non_transparent = [spear[np.argmax(spear!=2)] for spear in spears]
message = np.array(first_non_transparent).reshape((IMG_H, IMG_W))
for row in message:
    row = np.where(row==0, '.', row) 
    row = np.where(row=='1', 'X', row) 
    print("".join(row))
print()

XXXX.XXX..X..X..XX..XXX..
X....X..X.X..X.X..X.X..X.
XXX..X..X.X..X.X..X.X..X.
X....XXX..X..X.XXXX.XXX..
X....X....X..X.X..X.X.X..
X....X.....XX..X..X.X..X.

CPU times: user 3.25 ms, sys: 1.43 ms, total: 4.69 ms
Wall time: 3.42 ms


## Day 9: Sensor Boost

### setup

In [29]:
# GET LEVEL INPUT
boost_prog_raw = get_level_input("09")
boost_prog_list = list(map(int, boost_prog_raw.split(',')))
boost_prog = tuple(boost_prog_list)

### part one

In [30]:
%%time
BOOST_INPUT = 1

_, outputs = intcomp.run_computer(list(boost_prog)+[0 for x in range(9999)], [BOOST_INPUT])
print_result(outputs[0])

'RESULT: 2204990589'

'TIME............................................................'
CPU times: user 1.58 ms, sys: 131 µs, total: 1.71 ms
Wall time: 1.71 ms


### part two 

In [31]:
%%time
SENSOR_MODE = 2

_, outputs = intcomp.run_computer(list(boost_prog)+[0 for x in range(9999)], [SENSOR_MODE])
print_result(outputs[0])

'RESULT: 50008'

'TIME............................................................'
CPU times: user 1.18 s, sys: 8.55 ms, total: 1.19 s
Wall time: 1.2 s


## Day 10: Monitoring Station 

### setup 

In [32]:
asteroids = get_level_input("10").splitlines()
asteroid_grid = np.array([np.array(list(belt)) for belt in asteroids])
asteroid_set = frozenset(zip(*np.where(asteroid_grid == '#')[::-1]))

X = 0
Y = 1

sort_order = {
    '+': 1,
    '-': -1
}

def calc_slope(p1, p2):
    if(p1[X]-p2[X] == 0):
        return (np.inf, '+' if (p1[Y]-p2[Y]) < 0 else '-')
    if(p1[Y]-p2[Y]==0):
        return (0, '-' if (p1[X]-p2[X]) > 0 else '+')
    return ((p1[Y]-p2[Y])/(p1[X]-p2[X]), '-' if (p1[X]-p2[X]) > 0 else '+')

### part one

In [33]:
%%time

visible_sets = [(asteroid, set(map(calc_slope, itertools.repeat(asteroid), asteroid_set-{asteroid}))) for asteroid in asteroid_set]
  
monitoring_location, visible_asteroids = max(visible_sets, key = lambda x: len(x[1]))
print_result(f"{len(visible_asteroids)} asteroids visible at {monitoring_location}")

'RESULT: 284 asteroids visible at (20, 19)'

'TIME............................................................'
CPU times: user 299 ms, sys: 5.79 ms, total: 305 ms
Wall time: 307 ms


### part two 

In [34]:
%%time
MONITORING_STATION = (20, 19)
REMAINING_ASTEROIDS = asteroid_set-{MONITORING_STATION}

def l1(p1, p2):
    return abs(p1[X]-p2[X])+abs(p1[Y]-p2[Y])

def calc_slope_and_dist(p1, p2):
    dist = l1(p1,p2)
    if(p1[X]-p2[X] == 0):
        return (np.inf, '+' if (p1[Y]-p2[Y]) > 0 else '-', dist)
    if(p1[Y]-p2[Y]==0):
        return (0, '-' if (p1[X]-p2[X]) > 0 else '+', dist)
    return (-(p1[Y]-p2[Y])/(p1[X]-p2[X]), '-' if (p1[X]-p2[X]) > 0 else '+', dist)

visible_list = [(asteroid, calc_slope_and_dist(MONITORING_STATION, asteroid)) for asteroid in REMAINING_ASTEROIDS]
visible_list.sort(key=lambda k: (sort_order[k[1][1]], k[1][0], -k[1][2]), reverse=True)

visible_stacked = [list(g) for k, g in itertools.groupby(visible_list, key=lambda k: k[1][:1])]
two_hundredth_vape = visible_stacked[199][0][0]

output_val = two_hundredth_vape[0]*100+two_hundredth_vape[1]

print_result(output_val)

'RESULT: 404'

'TIME............................................................'
CPU times: user 2.63 ms, sys: 376 µs, total: 3 ms
Wall time: 2.66 ms


## Day 11: Space Police

### setup 

In [35]:
paint_prog = get_level_input("11")
paint_prog = list(map(int, paint_prog.split(',')))
paint_prog = tuple(paint_prog)

BLACK = 0
WHITE = 1
DIRECTIONS = ['^','>','v', '<']
DIR_MOVE = {
    '^': [0, 1],
    '>': [1, 0],
    'v': [0, -1],
    '<': [-1, 0]
}

ROTATE = {
    0: -1,
    1: 1
}

### part one 

In [45]:
%%time
robot_position = [0,0, '^']
white_set = set()
black_set = {(0,0)}

def get_color(rob_pos):
    if rob_pos in white_set:
        return WHITE
    return BLACK

def add_color(output, white_set, black_set, robot_position):
    cur_point = tuple(robot_position[:2])
    white_set-={cur_point}
    black_set-={cur_point}
    if(output==WHITE):
        white_set.add(cur_point)
    if(output==BLACK):
        black_set.add(cur_point)
        
def move_robot(output, robot_position):
    direction = robot_position[-1]
    dir_index = DIRECTIONS.index(direction)
    new_dir = DIRECTIONS[(dir_index+ROTATE[output])%len(DIRECTIONS)]
    return list(map(sum, zip(*[robot_position[:2], DIR_MOVE[new_dir]])))+[new_dir]
    
def run_robot(robot_position, white_set, black_set):
    paint_prog_ex = list(paint_prog)+[0 for _ in range(100000)]
    ip = 0
    while(True):
        current_color = get_color(tuple(robot_position[:2]))
        cur_input = [current_color for x in range(10)]
        ip, outputs = intcomp.run_computer(paint_prog_ex, cur_input, ip = ip, return_on_output=True)
        if outputs == []:
            break
        add_color(outputs[0], white_set, black_set, robot_position)
        ip, outputs = intcomp.run_computer(paint_prog_ex, cur_input, ip = ip, return_on_output=True)
        if outputs == []:
            break
        robot_position = move_robot(outputs[0], robot_position)
    return len(white_set)+len(black_set)

run_robot(robot_position, white_set, black_set)

CPU times: user 378 ms, sys: 2.95 ms, total: 381 ms
Wall time: 382 ms


2293

### part one 

In [47]:
%%time
robot_position = [0,0, '^']
white_set = {(0,0)}
black_set = set()

def get_color(rob_pos):
    if rob_pos in white_set:
        return WHITE
    return BLACK

def add_color(output, white_set, black_set, robot_position):
    cur_point = tuple(robot_position[:2])
    white_set-={cur_point}
    black_set-={cur_point}
    if(output==WHITE):
        white_set.add(cur_point)
    if(output==BLACK):
        black_set.add(cur_point)
        
def move_robot(output, robot_position):
    direction = robot_position[-1]
    dir_index = DIRECTIONS.index(direction)
    new_dir = DIRECTIONS[(dir_index+ROTATE[output])%len(DIRECTIONS)]
    return list(map(sum, zip(*[robot_position[:2], DIR_MOVE[new_dir]])))+[new_dir]
    
def run_robot(robot_position, white_set, black_set):
    paint_prog_ex = list(paint_prog)+[0 for _ in range(100000)]
    ip = 0
    while(True):
        current_color = get_color(tuple(robot_position[:2]))
        print(current_color)
        print(f"white set: {white_set}")
        print(f"black set: {black_set}")
        print(tuple(robot_position[:2]))
        print()
        cur_input = [current_color for x in range(10)]
        ip, outputs = intcomp.run_computer(paint_prog_ex, cur_input, ip = ip, return_on_output=True)
        if outputs == []:
            break
        print(f"FULL ROUND")
        print(f"input: {cur_input[0]}")
        print(f"paint_stage")
        print(f"output: {outputs}")
        add_color(outputs[0], white_set, black_set, robot_position)
        print(f"robot pos: {robot_position}")
        print(f"white set: {white_set}")
        print(f"black set: {black_set}")
        ip, outputs = intcomp.run_computer(paint_prog_ex, cur_input, ip = ip, return_on_output=True)
        if outputs == []:
            break
        print(f"move_stage")
        print(f"output: {outputs}")
        robot_position = move_robot(outputs[0], robot_position)
        print(f"robot pos: {robot_position}")
        print()
    return len(white_set)+len(black_set)

run_robot(robot_position, white_set, black_set)

1
white set: {(0, 0)}
black set: set()
(0, 0)

FULL ROUND
input: 1
paint_stage
output: [0]
robot pos: [0, 0, '^']
white set: set()
black set: {(0, 0)}
move_stage
output: [1]
robot pos: [1, 0, '>']

0
white set: set()
black set: {(0, 0)}
(1, 0)

FULL ROUND
input: 0
paint_stage
output: [0]
robot pos: [1, 0, '>']
white set: set()
black set: {(1, 0), (0, 0)}
move_stage
output: [1]
robot pos: [1, -1, 'v']

0
white set: set()
black set: {(1, 0), (0, 0)}
(1, -1)



KeyError: 36